In [1]:
import os 

os.chdir("../..")

from datasets import Dataset, load_dataset
from transformers import AutoTokenizer, AutoModelForSeq2SeqLM, TrainingArguments, Trainer, DataCollatorForSeq2Seq, AutoModelForSequenceClassification
from peft import get_peft_model, LoraConfig, TaskType
import evaluate
import numpy as np
from task1.config import ProjectPaths
import pandas as pd
import torch

paths = ProjectPaths()

# === 3. Set device ===
device = "mps" if torch.backends.mps.is_available() else "cpu"

# === 4. Load and preprocess data ===
def load_datasets(path):
    df = pd.read_csv(path, sep='\t')
    df = df[df['label'].isin(['SUBJ', 'OBJ'])].copy()
    df['label'] = df['label'].map({'OBJ': 0, 'SUBJ': 1})
    df = df[['sentence', 'label']]
    return Dataset.from_pandas(df)

train_dataset = load_datasets(paths.bulgarian_data_dir / "train_bg.tsv")
val_dataset   = load_datasets(paths.bulgarian_data_dir / "dev_bg.tsv")
test_dataset  = load_datasets(paths.bulgarian_data_dir / "dev_test_bg.tsv")

W0614 22:57:38.427000 37924 Lib\site-packages\torch\distributed\elastic\multiprocessing\redirects.py:29] NOTE: Redirects are currently not supported in Windows or MacOs.


In [2]:
model_name = "iarfmoose/roberta-base-bulgarian"
model = AutoModelForSequenceClassification.from_pretrained(model_name, num_labels=2)
tokenizer = AutoTokenizer.from_pretrained(model_name)

config.json:   0%|          | 0.00/516 [00:00<?, ?B/s]

d:\Master\AN1SEM2\BioNLP\CLEF2025-CheckThat\task1\.venv\Lib\site-packages\huggingface_hub\file_download.py:143: UserWarning: `huggingface_hub` cache-system uses symlinks by default to efficiently store duplicated files but your machine does not support them in C:\Users\Asus\.cache\huggingface\hub\models--iarfmoose--roberta-base-bulgarian. Caching files will still work but in a degraded version that might require more space on your disk. This warning can be disabled by setting the `HF_HUB_DISABLE_SYMLINKS_WARNING` environment variable. For more details, see https://huggingface.co/docs/huggingface_hub/how-to-cache#limitations.
To support symlinks on Windows, you either need to activate Developer Mode or to run Python as an administrator. In order to activate developer mode, see this article: https://docs.microsoft.com/en-us/windows/apps/get-started/enable-your-device-for-development
  warnings.warn(message)


pytorch_model.bin:   0%|          | 0.00/507M [00:00<?, ?B/s]

Some weights of RobertaForSequenceClassification were not initialized from the model checkpoint at iarfmoose/roberta-base-bulgarian and are newly initialized: ['classifier.dense.bias', 'classifier.dense.weight', 'classifier.out_proj.bias', 'classifier.out_proj.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


tokenizer_config.json:   0%|          | 0.00/78.0 [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/1.79M [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/1.44M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/239 [00:00<?, ?B/s]

In [3]:
def tokenize_fn(examples):
    return tokenizer(
        examples["sentence"],
        padding="max_length",
        truncation=True,
        max_length=128
    )

train_dataset = train_dataset.map(tokenize_fn, batched=True)
val_dataset = val_dataset.map(tokenize_fn, batched=True)
test_dataset = test_dataset.map(tokenize_fn, batched=True)

train_dataset = train_dataset.rename_column("label", "labels")
val_dataset = val_dataset.rename_column("label", "labels")
test_dataset = test_dataset.rename_column("label", "labels")

train_dataset.set_format(type="torch", columns=["input_ids", "attention_mask", "labels"])
val_dataset.set_format(type="torch", columns=["input_ids", "attention_mask", "labels"])
test_dataset.set_format(type="torch", columns=["input_ids", "attention_mask", "labels"])

Map:   0%|          | 0/691 [00:00<?, ? examples/s]

Map:   0%|          | 0/306 [00:00<?, ? examples/s]

Map:   0%|          | 0/250 [00:00<?, ? examples/s]

In [4]:
lora_config = LoraConfig(
    r=16,
    lora_alpha=32,
    lora_dropout=0.05,
    task_type=TaskType.SEQ_CLS,
    target_modules=["query_proj", "key_proj", "value_proj", "dense"] 
)

model = get_peft_model(model, lora_config).to(device)

In [5]:
f1 = evaluate.load("f1")
accuracy = evaluate.load("accuracy")
precision = evaluate.load("precision")
recall = evaluate.load("recall")

def compute_metrics(eval_pred):
    logits, labels = eval_pred
    preds = np.argmax(logits, axis=-1)
    return {
        "accuracy": accuracy.compute(predictions=preds, references=labels)["accuracy"],
        "f1_macro": f1.compute(predictions=preds, references=labels, average="macro")["f1"],
        "precision": precision.compute(predictions=preds, references=labels, average="macro")["precision"],
        "recall": recall.compute(predictions=preds, references=labels, average="macro")["recall"],
    }

In [6]:
# === 8. TrainingArguments ===
training_args = TrainingArguments(
    output_dir="./results/bulgarian-lora",
    per_device_train_batch_size=4,
    per_device_eval_batch_size=4,
    num_train_epochs=10,
    eval_strategy="epoch",
    save_strategy="epoch",
    logging_steps=10,
    load_best_model_at_end=True,
    metric_for_best_model="f1_macro",
)

In [7]:
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset,
    eval_dataset=val_dataset,
    compute_metrics=compute_metrics
)

# === 10. Train ===
trainer.train()

# === 11. Evaluate on test set ===
trainer.evaluate(eval_dataset=test_dataset)

  0%|          | 0/1730 [00:00<?, ?it/s]

d:\Master\AN1SEM2\BioNLP\CLEF2025-CheckThat\task1\.venv\Lib\site-packages\torch\utils\data\dataloader.py:665: UserWarning: 'pin_memory' argument is set as true but no accelerator is found, then device pinned memory won't be used.
  warnings.warn(warn_msg)


{'loss': 0.7169, 'grad_norm': 6.391135215759277, 'learning_rate': 4.971098265895954e-05, 'epoch': 0.06}
{'loss': 0.7372, 'grad_norm': 4.695607662200928, 'learning_rate': 4.942196531791908e-05, 'epoch': 0.12}
{'loss': 0.6815, 'grad_norm': 4.262930393218994, 'learning_rate': 4.913294797687861e-05, 'epoch': 0.17}
{'loss': 0.7095, 'grad_norm': 8.437414169311523, 'learning_rate': 4.8843930635838154e-05, 'epoch': 0.23}
{'loss': 0.639, 'grad_norm': 5.070406436920166, 'learning_rate': 4.855491329479769e-05, 'epoch': 0.29}
{'loss': 0.6745, 'grad_norm': 7.1451096534729, 'learning_rate': 4.826589595375723e-05, 'epoch': 0.35}
{'loss': 0.6445, 'grad_norm': 5.549628734588623, 'learning_rate': 4.7976878612716764e-05, 'epoch': 0.4}
{'loss': 0.6632, 'grad_norm': 7.040141582489014, 'learning_rate': 4.7687861271676305e-05, 'epoch': 0.46}
{'loss': 0.6615, 'grad_norm': 5.789669513702393, 'learning_rate': 4.739884393063584e-05, 'epoch': 0.52}
{'loss': 0.6017, 'grad_norm': 4.460951805114746, 'learning_rate':

  0%|          | 0/77 [00:00<?, ?it/s]

{'eval_loss': 0.5747998952865601, 'eval_accuracy': 0.7124183006535948, 'eval_f1_macro': 0.707913918778202, 'eval_precision': 0.7104191302062803, 'eval_recall': 0.7069745401283763, 'eval_runtime': 16.7972, 'eval_samples_per_second': 18.217, 'eval_steps_per_second': 4.584, 'epoch': 1.0}


...

d:\Master\AN1SEM2\BioNLP\CLEF2025-CheckThat\task1\.venv\Lib\site-packages\huggingface_hub\file_download.py:943: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(


{'train_runtime': 2534.6994, 'train_samples_per_second': 2.726, 'train_steps_per_second': 0.683, 'train_loss': 0.42331261124914094, 'epoch': 10.0}


d:\Master\AN1SEM2\BioNLP\CLEF2025-CheckThat\task1\.venv\Lib\site-packages\torch\utils\data\dataloader.py:665: UserWarning: 'pin_memory' argument is set as true but no accelerator is found, then device pinned memory won't be used.
  warnings.warn(warn_msg)


  0%|          | 0/63 [00:00<?, ?it/s]

{'eval_loss': 0.8820622563362122,
 'eval_accuracy': 0.744,
 'eval_f1_macro': 0.7385791778315143,
 'eval_precision': 0.7385791778315143,
 'eval_recall': 0.7385791778315143,
 'eval_runtime': 14.1982,
 'eval_samples_per_second': 17.608,
 'eval_steps_per_second': 4.437,
 'epoch': 10.0}